---

# 🛠️ Tools in AI: Give a Bot a Tool, Change Its Life

---

## 🎯 Why Tools Matter

Think of it like this:

> **Give a bot an answer, it solves one problem.  
> Teach a bot to use tools, it solves problems for life.**

In this lab, we’re not just feeding the AI information —  
we’re **handing it a toolkit** so it can **fetch data**, **call APIs**, **take actions**, and **be truly useful**.

This is how we level up from **"chatbot"** ➔ **"real-world assistant."**

---

## 🔥 The Binford Tools Moment

Just like Tim "The Toolman" Taylor from *Binford Tools* —  
**more power** makes everything better (and sometimes louder).

In AI, *tools = more power*:

| Without Tools | With Tools |
|:---|:---|
| "I don't know, sorry." | "Let me fetch that for you!" |
| Static chatbot | Dynamic action-taker |
| Limited answers | Internet searches, database queries, transactions |

Our goal is to turn your bots from polite talkers ➔ action heroes — **with Binford-grade upgrades.**

---

## 🧩 Example

Imagine your chatbot can:

- Search a website for the latest news  
- Pull a customer record from a database  
- Book an appointment directly from the conversation

✅ It’s not guessing.  
✅ It’s **doing.**

---

# 🎯 Final Thought

> **A chatbot with tools isn't just smart — it's useful.**  
> And just like Binford Tools... we’re here for **more power!**

---


<img src="../binford.png"/>

In [ ]:
# 🛠️ Setup: Imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [ ]:
# 🛠️ Step 1: Load environment variables
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key loaded: {openai_api_key[:8]}...")
else:
    print("OpenAI API Key not found. Please check your .env file.")

# 🛠️ Step 2: Initialize OpenAI client
openai = OpenAI(api_key=openai_api_key)


In [ ]:
# 🛠️ Step 3: System Message — Sets the behavior of the AI
system_message = """You are a helpful assistant for an airline called FlightAI.
You can help users with flight prices and timings. For example, if a user says 'planning to travel to London give me price and timing, preferably early', 
you should provide the ticket price and the earliest flight timing available."""

In [ ]:
# 🛠️ Step 4: Define Tool Functions (for Ticket Prices & Flight Timings)
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to a destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "City the customer wants to travel to"
            }
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

time_function = {
    "name": "get_flight_time",
    "description": "Get the flight times for a destination city based on AM/PM preference.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "City the customer wants to travel to"
            },
            "travel_time": {
                "type": "string",
                "description": "Preferred time of day to travel (AM/PM)"
            }
        },
        "required": ["destination_city", "travel_time"],
        "additionalProperties": False
    }
}

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": time_function}
]



In [ ]:
# 🛠️ Step 5: Define Tool Behavior (what each tool actually does)
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

flight_times = {
    "london": {"am": ["9", "11"], "pm": ["6", "9"]},
    "paris": {"am": ["8", "10"], "pm": ["5", "8"]},
    "tokyo": {"am": ["7", "9"], "pm": ["4", "7"]},
    "berlin": {"am": ["6", "8"], "pm": ["3", "6"]}
}

def get_ticket_price(destination_city):
    print(f"Calling get_ticket_price for {destination_city}")
    return ticket_prices.get(destination_city.lower(), "Unknown")

def get_flight_time(destination_city, travel_time):
    print(f"Calling get_flight_time for {destination_city} during {travel_time}")
    city_info = flight_times.get(destination_city.lower())
    if not city_info:
        return "Unknown"
    times = city_info.get(travel_time.lower(), "Unknown")
    if times == "Unknown":
        return "Unknown"
    return " ".join([time + travel_time.lower() for time in times])

In [ ]:
# 🛠️ Step 6: Handle when the AI wants to use a tool
def handle_tool_call(message):
    responses = []  # We'll collect the tool responses here

    # Loop through all the tool calls the AI made
    for tool_call in message.tool_calls:
        # Parse the arguments the AI sent (they come in JSON format)
        arguments = json.loads(tool_call.function.arguments)

        # Extract the destination city from the arguments
        city = arguments.get('destination_city')

        # 🎯 Figure out which tool the AI is asking for
        if tool_call.function.name == "get_ticket_price":
            # If AI wants ticket price, call get_ticket_price()
            price = get_ticket_price(city)
            response_content = {
                "destination_city": city,
                "price": price
            }
        
        elif tool_call.function.name == "get_flight_time":
            # If AI wants flight times, call get_flight_time()
            travel_time = arguments.get('travel_time')
            flight_time_str = get_flight_time(city, travel_time)
            response_content = {
                "destination_city": city,
                "flight_times": flight_time_str
            }
        
        else:
            # If AI asked for a tool we don't recognize
            response_content = {
                "error": "Unknown tool call"
            }

        # Build a response the AI will understand
        responses.append({
            "role": "tool",
            "content": json.dumps(response_content),
            "tool_call_id": tool_call.id  # Must match the tool_call id so the AI knows which tool answered
        })

    # Return:
    # - List of tool responses to send back to the AI
    # - List of all destination cities involved (might be useful later)
    return responses, [json.loads(tc.function.arguments).get('destination_city') for tc in message.tool_calls]


In [ ]:
# 🛠️ Step 7: Chat logic
def chat_flight_ai(message, history=None):
    # If this is the user's first message, initialize an empty history
    if history is None:
        history = []

    # Build the conversation history:
    # Start with the system message, then add previous conversation history, then the new user message
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    # Call the OpenAI API to get a response
    # We pass the message history and the list of tools the AI can use
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools
    )

    # Debug print — shows what the AI is thinking/responding with
    print(response.choices[0])

    # 🎯 Handle Tool Calls if needed
    # If the AI’s "finish_reason" says "tool_calls", it means the AI wants to use a tool
    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message  # Get the AI's tool request (not final answer yet)
        
        # Handle the tool call: call the actual Python function
        tool_responses, _ = handle_tool_call(message)
        
        # Add the tool call and the tool's response to the conversation history
        messages.append(message)           # What the AI *asked* (e.g., "I want ticket price for London")
        messages.extend(tool_responses)    # What our tool *returned* (e.g., "The ticket price is $799")
        
        # Re-send updated messages to the AI
        # Now that the tool gave answers, the AI can continue the conversation properly
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )

    # Finally, when AI finishes (no more tool calls needed), return:
    # - The final AI message (what it says to the user)
    # - The updated full conversation history (for next turn)
    return response.choices[0].message.content, messages

In [ ]:
# 🛠️ Step 8: Launch Gradio Interface
def launch_gr():
    gr.ChatInterface(fn=chat_flight_ai, type="messages").launch()

In [ ]:
# 🛠️ Step 9: Run the app
if __name__ == "__main__":
    launch_gr()


# 🛠️ Lab - Add New Tool: Suggest an Airline

## ✈️ What it Does

When a customer says something like:  
*"Which airline can I take to London?"*  
your bot can call this new tool to suggest an airline flying to that destination.

---

# 🔥 New Tool Definition

```python
# Define suggest_airline tool
airline_function = {
    "name": "suggest_airline",
    "description": "Suggest an airline that flies to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city the customer wants to travel to"
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}
```

---

# 🛠️ Tool Behavior (the Python function)

```python
# Airlines database
airlines = {
    "london": "British Airways",
    "paris": "Air France",
    "tokyo": "All Nippon Airways",
    "berlin": "Lufthansa"
}

# Tool function to suggest airline
def suggest_airline(destination_city):
    print(f"Calling suggest_airline for {destination_city}")
    return airlines.get(destination_city.lower(), "No airline found")
```

---

# 📦 Update Your Tools List

Now add it to your `tools` list:

```python
tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": time_function},
    {"type": "function", "function": airline_function}  # <-- new addition!
]
```

---

# 🛠️ Update handle_tool_call

Extend your `handle_tool_call(message)` function to recognize it:

```python
elif tool_call.function.name == "suggest_airline":
    airline = suggest_airline(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city, "airline": airline}),
        "tool_call_id": tool_call.id
    }
```

---

# 🎯 Summary

| Tool | Action |
|:---|:---|
| `get_ticket_price` | Return the ticket price |
| `get_flight_time` | Return flight timings (AM/PM) |
| `suggest_airline` | Return a suggested airline |


---

# 🏆 🎮 Achievement Unlocked: **Master of Tools**

```
────────────────────────────────────────────
⚙️ Tool Lab Completed!

🎯 New Powers Gained:
- Summon ticket prices out of thin air
- Bend flight times to your will
- Suggest airlines like a seasoned travel agent

🛡️ New Title: Tool Tamer

🚀 Next Quest: Build even smarter AIs that take action like pros
────────────────────────────────────────────
```

---